In [1]:
import os
import json
import cv2
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile
import os

zip_file = '/content/drive/MyDrive/Data/Skin_Phone.zip'
output_dir = '/content/skin_phone_data'

try:
    # .egg 파일을 zip으로 열어보기
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"압축 해제 완료: {output_dir}")
except zipfile.BadZipFile:
    print(".egg 파일이 일반 zip 형식이 아닙니다.")

압축 해제 완료: /content/skin_phone_data


In [4]:
# 데이터가 저장된 최상위 폴더 경로
base_dir = "/content/skin_phone_data/Skin_Phone/Train/Smart_Phone_Image"

In [ ]:
# 결과를 저장할 딕셔너리
image_data = {}  # {폴더명: {이미지파일명: OpenCV 이미지 객체}}

# 폴더 순회
for idx, folder_name in enumerate(os.listdir(base_dir)):
    folder_path = os.path.join(base_dir, folder_name)

    # 폴더인지 확인
    if os.path.isdir(folder_path):
        if idx % 100 == 0:
            print(f"Processing folder: {folder_name}")

        # 폴더별 이미지 저장
        folder_images = {}

        # 폴더 내 파일 탐색
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            # 이미지 파일 처리 (.jpg 확장자)
            if file_name.endswith(".jpg"):
                try:
                    # 한글 경로 처리: 바이너리 읽기 후 OpenCV로 디코딩
                    with open(file_path, 'rb') as f:
                        file_data = np.asarray(bytearray(f.read()), dtype=np.uint8)
                        image = cv2.imdecode(file_data, cv2.IMREAD_COLOR)

                    # 이미지 로드 및 저장
                    if image is not None:
                        # RGB2BGR : 머신러닝에 색이 영향을 미치진 않을 것 같음
                        # img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                        # folder_images[file_name] = img_rgb

                        folder_images[file_name] = image
                    else:
                        print(f"Failed to load image: {file_path}")
                except Exception as e:
                    print(f"Error loading image {file_path}: {e}")

        # 폴더의 이미지 데이터를 최종 딕셔너리에 추가
        image_data[folder_name] = folder_images

print("Image loading complete!")


Processing folder: 1090
Processing folder: 0875
Processing folder: 0883


In [ ]:
#데이터 확인
print(image_data.keys())
print(image_data['0002'].keys())
print()
print(image_data['0002']['0002_03_L.jpg'].shape)

In [ ]:
#경로지정
base_dir = "/content/skin_phone_data/Skin_Phone_Small/Train/Smart_Phone_Label"

In [ ]:
#제이슨파일 저장할 딕셔너리 생성
filtered_json_data = {}

In [ ]:
# 지정한 경로에 있는 파일들을 가져옴
for forder_name in os.listdir(base_dir):
    #기본경로에 위에가져온 파일이름을 붙여서 새로운 경로 작성
    forder_path = os.path.join(base_dir, forder_name)
    #폴더인지 확인하는 과정
    if os.path.isdir(forder_path):
        print(f"processing folder : {forder_name}")

        #폴더내 라벨링 데이터 저장할 공간 생성
        forder_jsons = {}

        #폴더를 순회하며 파일 검색
        for file_name in os.listdir(forder_path):

            #파일 경로 합치기
            file_path = os.path.join(forder_path, file_name)

            #01.json으로 끝나는 파일 찾기
            if file_name.endswith("01.json"):
                try:
                    with open(file_path, "r", encoding= "utf-8") as f:
                        json_data = json.load(f)

                    forder_jsons[file_name] = json_data
                except Exception as e:
                    print(f"Error {file_path} : {e}")

        if forder_jsons:
            filtered_json_data[forder_name] = forder_jsons

In [ ]:
#데이터 확인 제이슨파일
print(filtered_json_data.keys())
print(filtered_json_data["0002"])
print()
print(filtered_json_data['0002']['0002_03_R_01.json']['images']['bbox'])

In [ ]:
#데이터 확인 이미지파일
print(image_data.keys())
print(image_data['0002'].keys())
print()
print(image_data['0002']['0002_03_L.jpg'].shape)

In [ ]:
import numpy as np
pigment = []
for human_id in filtered_json_data.keys():
    for file_name in filtered_json_data[human_id].keys():
        pigment.append(filtered_json_data[human_id][file_name]['annotations']['forehead_pigmentation'])
print(np.unique(pigment))

In [ ]:
cropped_image_dict = {} # 저장받을 딕셔너리 생성

for human_id in filtered_json_data.keys(): # 사람번호 조회 0002, 0004 등

    for file_name in filtered_json_data[human_id].keys(): # 파일이름 조회 (제이슨파일)

        bbox = filtered_json_data[human_id][file_name]['images']['bbox'] #조회된 파일의 비박스정보 가져오기
        x1, y1, x2, y2 = bbox #비박스 데이터 나누기
        image_file_name = filtered_json_data[human_id][file_name]["info"]["filename"] #해당 제이슨 파일의 이미지파일 정보조회
        image = image_data[human_id][image_file_name] #이미지 불러오기
        cropped_image = image[y1:y2, x1:x2] #불러온 이미지 위에 저장한 비박스로 자르기
        cropped_image_dict[image_file_name] = cropped_image #자른이미지 딕셔너리에 이름과 3차원데이터로 저장하기

In [ ]:
# 이미지 및 바운딩 박스 정보 딕셔너리 구성 확인
print(cropped_image_dict.keys())
print(len(cropped_image_dict.keys()))
print(cropped_image_dict['0009_03_R.jpg'].shape)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#잘린 이미지 파일 확인
cropped_image_file_name = cropped_image_dict.keys() # 파일명 가져오기
image_list = list(cropped_image_file_name) # 슬라이싱을 위해 리스트로 변경


for idx, cin in enumerate(image_list[:9]):
    plt.subplot(3, 3, idx+1)
    plt.imshow(cropped_image_dict[cin])
    plt.title(cin)
    plt.axis("off")

plt.show()